In [1]:
import langchain_community
import os

# Load Data

In [2]:
from langchain_community.document_loaders import DirectoryLoader

from dotenv import load_dotenv

load_dotenv(override=True)


# loader = DirectoryLoader("data", glob="*.md", show_progress=True)
# docs = loader.load()

# if docs:
# 	print(docs[0].page_content[:100])
# else:
# 	print("No documents found.")



True

# RAG Framework

chunking & emdedding 

In [3]:
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import Document

import shutil


CHROMA_PATH = "chroma"
from langchain.schema import Document

combined_docs_path = "combined_documents.md"
with open(combined_docs_path, "r", encoding="utf-8") as f:
    content = f.read()

combined_docs = [Document(page_content=content, metadata={"source": combined_docs_path})]

# chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(combined_docs)

# embedding
if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

vectorstore = Chroma.from_documents(
        chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH
)

print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")


Saved 411 chunks to chroma.


invoking rag

In [4]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

llm = ChatOpenAI(model="gpt-4o-mini")

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print(retriever | format_docs)
print(retriever)
print(format_docs)


rag_chain.invoke("What is Task Decomposition?")

e:\Repositories\graphrag-businessqa-evaluation\env\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


first=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000186D09030E0>, search_kwargs={}) middle=[] last=RunnableLambda(format_docs)
tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000186D09030E0> search_kwargs={}
<function format_docs at 0x00000186CD955800>


'Task Decomposition refers to breaking down detailed work activities into more manageable tasks for analysis. This process helps assess the technical automation potential of various occupations by analyzing their component activities. It allows for a clearer understanding of the performance capabilities required for each task.'

# Preparing Evaluation

In [14]:
import json

# Load synthetic data
with open('synthetic_data_big_context.json', 'r') as f:
    synthetic_data = json.load(f)

# Access the list under the "responses" key
if isinstance(synthetic_data, dict) and 'responses' in synthetic_data:
    synthetic_data = synthetic_data['responses']

# Extract questions, ground_truth, and context
questions = [item.get('question', "") for item in synthetic_data]
ground_truths = [item.get('ground_truth', "") for item in synthetic_data]

answers = []
contexts = []

# Print to verify mappings
print("Questions:", questions)
print("Ground Truths:", ground_truths)



Questions: ['What is the estimated economic impact of generative AI on the global economy?', 'Which business functions could see the most value from generative AI?', 'How will generative AI affect the future of work?', 'What is the projected timeline for the automation of work activities due to generative AI?', 'What are the key challenges associated with the adoption of generative AI?']
Ground Truths: ['Generative AI could add the equivalent of $2.6 trillion to $4.4 trillion annually across the 63 use cases analyzed.', 'About 75 percent of the value that generative AI use cases could deliver falls across four areas: Customer operations, marketing and sales, software engineering, and R&D.', 'Generative AI has the potential to change the anatomy of work, augmenting the capabilities of individual workers by automating some of their individual activities.', 'Estimates suggest that half of today’s work activities could be automated between 2030 and 2060, with a midpoint in 2045.', 'Challen

In [22]:
from datasets import Dataset

# Clear previous results
answers.clear()
contexts.clear()

# Inference
for query in questions:
    answer = rag_chain.invoke(query)
    print(answer)
    context = [doc.page_content for doc in retriever.get_relevant_documents(query)]
    answers.append(answer)
    contexts.append(context)

print("Answers:", answers)
# Prepare data
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

print(data)

# Convert to dataset
dataset = Dataset.from_dict(data)

The estimated economic impact of generative AI on the global economy is projected to be between $2.6 trillion and $4.4 trillion annually. This represents an increase of 15 to 40 percent in the overall impact of artificial intelligence. The potential impact could roughly double when considering its integration into existing software used for other tasks.
The business functions that could see the most value from generative AI are customer operations, marketing and sales, software engineering, and research and development. These areas are estimated to account for approximately 75 percent of the total annual value generated by AI use cases. Generative AI can enhance interactions, create marketing content, and assist in software development among other applications.
Generative AI is expected to significantly change the future of work by automating 60 to 70 percent of tasks that currently consume employees' time. Its enhanced ability to understand natural language will particularly benefit k

Evaluation 

In [25]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

Evaluating: 100%|██████████| 10/10 [00:14<00:00,  1.40s/it]


# Results

In [32]:
df

,user_input,retrieved_contexts,response,faithfulness,answer_relevancy
0,What is the estimated economic impact of gener...,[1. Generative AI’s impact on productivity cou...,The estimated economic impact of generative AI...,1.0,1.000000
1,Which business functions could see the most va...,[2. About 75 percent of the value that generat...,The business functions that could see the most...,1.0,0.981748
2,How will generative AI affect the future of work?,[4. Generative AI has the potential to change ...,Generative AI is expected to significantly cha...,1.0,0.977863
3,What is the projected timeline for the automat...,[- The capabilities of generative AI have incr...,The projected timeline for automating work act...,1.0,0.996527
4,What are the key challenges associated with th...,[# The economic potential of generative AI: Th...,The key challenges associated with the adoptio...,1.0,1.000000
